In [5]:
[(i,j) for i in range(3) for j in range(2)]

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

In [146]:
from copy import deepcopy

ROWS=2
COLUMNS=3

class GameState:

    def __init__(self):
        """At a minimum, the board state needs to keep track
        of which cells are open and closed; which player has 
        initiative (whose turn it is to move); and where each player 
        is on the board. 
        (Note: Remember to block off the lower right corner when you create a new board!)
        """
        # represent the board as a list of length = r*c
        self._board=[[0]*COLUMNS for _ in range(ROWS)]
        self._board[-1][-1]=1
        self._player=1
        self._positions=[None,None]
    
    def forecast_move(self, move):
        """ Return a new board object with the specified move
        applied to the current game state.
        
        Parameters
        ----------
        move: tuple
            The target position for the active player's next move
        """
        # Check if the move is in the legal open spaces on the board if not throw exception
        if move not in self.get_legal_moves():
            raise RuntimeError("Not a legal move!")
        # Deep copy this board object
        newboard=deepcopy(self)
        # place the current player at the position of the move
        newboard._positions[self._player]=move
        # toggle the player id
        if newboard._player == 1:
            newboard._player=0
        else:
            newboard._player=1
        # mark the move positon on the board as occupied 
        newboard._board[move[1]][move[0]]=1
        return newboard
    
    def get_legal_moves(self):
        """ Return a list of all legal moves available to the
        active player.  Each player should get a list of all
        empty spaces on the board on their first move, and
        otherwise they should get a list of all open spaces
        in a straight line along any row, column or diagonal
        from their current position. (Players CANNOT move
        through obstacles or blocked squares.) Moves should
        be a pair of integers in (column, row) order specifying
        the zero-indexed coordinates on the board.
        """
        loc = self._positions[self._player]
        if not loc:
            return self._get_open_spaces()
        # start with an empty moves list
        moves=[]
        # define a list of possible moves L,R,U,D,DDR,DDL,DUR,DUL
        dirs = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,1),(1,-1),(-1,-1)]
        # iterate through all of the possible directions from each position
        for dx,dy in dirs:
            _x,_y=loc
            # check to see that we do not iterate off the board
            while 0 <= _x+dx < ROWS and 0 <= _y+dy< COLUMNS:
                # Move along the current direction
                _x,_y=_x+dx,_y+dy
                # check if that square of the board is occupied
                if self._board[_x][_y]:
                    break
                moves.append((_y,_x))
        return moves
    
    def _get_open_spaces(self):
        # return the list of open board spaces i.e. not 1
        return [(y,x) for x in range(ROWS) for y in range(COLUMNS) if self._board[x][y]!=1]

In [147]:
tmp=GameState()

In [148]:
tmp._board

[[0, 0, 0], [0, 0, 1]]

In [149]:
tmp.get_legal_moves()

[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1)]

In [150]:
tmp._board[0][0]=1
tmp._board[0][1]=1

In [151]:
tmp._positions[0]=(0,0)
tmp._positions[1]=(0,1)


In [152]:
tmp.get_legal_moves()

[(1, 1), (2, 0), (0, 1)]

In [153]:
tmp.forecast_move((2,0))

In [164]:

def terminal_test(gameState):
    """ Return True if the game is over for the active player
    and False otherwise.
    """
    if len(gameState.get_legal_moves()) == 0:
        return True
    else:
        return False


def min_value(gameState):
    """ Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return 1
    v = float("inf")
    moves = gameState.get_legal_moves()
    for m in moves:
        v = min(v,max_value(gameState.forecast_move(m)))
    return v

def max_value(gameState):
    """ Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return -1
    v = float("-inf")
    moves = gameState.get_legal_moves()
    for m in moves:
        v = max(v,min_value(gameState.forecast_move(m)))
    return v

In [162]:
max_value(tmp)

inf

In [165]:
def minimax_decision(gameState):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    return max(gameState.get_legal_moves(),
               key=lambda m: min_value(gameState.forecast_move(m)))